## code with Binary Encoded

In [71]:
import random

# Declaring and initiallizing Parameters
NUM_THEORY_COURSES = 10 
NUM_LAB_COURSES = 3 
NUM_SECTIONS_PER_COURSE = 5 
NUM_PROFESSORS = 10 
NUM_DAYS = 5  
NUM_TIME_SLOTS_PER_DAY = 6 
CLASSROOM_CAPACITY = 60
LARGE_HALL_CAPACITY = 120
NUM_FLOORS = 3  
NUM_ROOMS_PER_FLOOR = 10 

TOTAL_ROOMS = NUM_FLOORS * NUM_ROOMS_PER_FLOOR

#Generating time slots according to required description
TIME_SLOTS = [
    ("8:30", "9:50"),
    ("10:05", "11:25"),
    ("11:40", "1:00"),
    ("1:15", "2:35"),
    ("2:50", "4:10"),
    ("4:25", "5:45")
]

# Define durations for theory and lab classes
THEORY_CLASS_DURATION = 1 # Assigning slot which is equal to 1:20
LAB_CLASS_DURATION = 2  # Assining 2 slots

# Define the maximum number of courses per professor and per section
MAX_COURSES_PER_PROFESSOR = 3
# Define the minimum number of days between lectures for the same course
MIN_DAYS_BETWEEN_LECTURES = 1
# Define the maximum number of floors traversed by teachers/students
MAX_FLOORS_TRAVERSED = 1
# Define the mutation rate
MUTATION_RATE = 0.1
# Define class types
THEORY_CLASS = "Theory"
LAB_CLASS = "Lab"
# Define room types
CLASSROOM = "Classroom"
LARGE_HALL = "Large Hall"

def chromosome_generation():
    chromosome = []
    for course_id in range(NUM_THEORY_COURSES):
        for section_id in range(NUM_SECTIONS_PER_COURSE):
            # Assign course type (Theory/Lab)
            course_type = THEORY_CLASS if random.random() < 0.25 else LAB_CLASS
            # Assign professor
            professor_id = random.randint(0, NUM_PROFESSORS - 1)
            # Assign first lecture day
            first_lecture_day = random.randint(0, NUM_DAYS - 1)
            # Ensure second lecture is at least one day after the first lecture
            second_lecture_day = (first_lecture_day + MIN_DAYS_BETWEEN_LECTURES + 1) % NUM_DAYS
            # Assign classroom and size based on course type
            if course_type == THEORY_CLASS:
                room_type = CLASSROOM
                room_capacity = CLASSROOM_CAPACITY
            else:
                room_type = LARGE_HALL
                room_capacity = LARGE_HALL_CAPACITY
            floor_number = random.randint(1, NUM_FLOORS)
            room_number_in_floor = random.randint(1, NUM_ROOMS_PER_FLOOR)
            room_number = (floor_number - 1) * NUM_ROOMS_PER_FLOOR + room_number_in_floor
            # Encode attributes into binary format
            binary_course = format(course_id, f'0{NUM_THEORY_COURSES.bit_length()}b')
            binary_section = format(section_id, f'0{NUM_SECTIONS_PER_COURSE.bit_length()}b')
            binary_professor = format(professor_id, f'0{NUM_PROFESSORS.bit_length()}b')
            binary_day1 = format(first_lecture_day, f'0{NUM_DAYS.bit_length()}b')
            binary_day2 = format(second_lecture_day, f'0{NUM_DAYS.bit_length()}b')
            binary_room = format(room_number, f'0{TOTAL_ROOMS.bit_length()}b')
            binary_room_capacity = format(room_capacity, f'0{max(CLASSROOM_CAPACITY, LARGE_HALL_CAPACITY).bit_length()}b')
            # Add binary-encoded chromosome information
            chromosome.append((binary_course, binary_section, course_type, binary_professor,
                               binary_day1, binary_day2, binary_room, binary_room_capacity))
    return chromosome

def fitness_Calculation(chromosome):
    conflicts = 0
    professor_schedule = {}
    room_schedule = {}
    section_schedule = {}
    section_course_count = {}

    daily_schedule = [[[] for _ in range(NUM_TIME_SLOTS_PER_DAY)] for _ in range(NUM_DAYS)]
    professor_courses_count = {professor_id: 0 for professor_id in range(NUM_PROFESSORS)}

    for course_info in chromosome:
        binary_course, binary_section, course_type, binary_professor, binary_day1, binary_day2, binary_room, binary_room_capacity = course_info
        # Decode binary values to integers
        course_id = int(binary_course, 2)
        section_id = int(binary_section, 2)
        professor_id = int(binary_professor, 2)
        day1 = int(binary_day1, 2)
        day2 = int(binary_day2, 2)
        room_number = int(binary_room, 2)
        room_capacity = int(binary_room_capacity, 2)

        # Calculate time slot indices
        time_slot_index1 = day1 * NUM_TIME_SLOTS_PER_DAY // NUM_DAYS
        time_slot_index2 = day2 * NUM_TIME_SLOTS_PER_DAY // NUM_DAYS

        # Check conflicts
        # 1. Check whether room is already occupied or not
        if (day1, room_number) in room_schedule or (day2, room_number) in room_schedule:
            conflicts += 1
        # 2. Check whether room chosen will contain all the registered students or not
        if (day1, time_slot_index1) in room_schedule and room_schedule[(day1, time_slot_index1)][1] != room_number:
            conflicts += 1
        if (day2, time_slot_index2) in room_schedule and room_schedule[(day2, time_slot_index2)][1] != room_number:
            conflicts += 1
        # 3. Professor will be assigned to only one section at a time for teaching
        if (professor_id, day1) in professor_schedule or (professor_id, day2) in professor_schedule:
            conflicts += 1
        # 4. Check whether the same section is assigned to two different rooms at the same time.
        if (day1, room_number) in section_schedule or (day2, room_number) in section_schedule:
            conflicts += 1
        # 5. Check if the professor is already assigned to the maximum number of courses
        if professor_courses_count[professor_id] >= MAX_COURSES_PER_PROFESSOR:
            conflicts += 1
        else:
            # Increment the count of courses for the professor
            professor_courses_count[professor_id] += 1
        # 6. Ensure there is at least one day difference between lecture days same course have not lecture in adjacent days
        if day1 == day2 or abs(day1 - day2) == 1:
            conflicts += 1
        # 7. No section can have more than 5 courses in a semester.
        if (course_id, section_id) in section_course_count:
            section_course_count[(course_id, section_id)] += 1
            if section_course_count[(course_id, section_id)] > 5:
                conflicts += 1
        else:
            section_course_count[(course_id, section_id)] = 1
        # 8. Lab lectures should be conducted in two consecutive slots.
        if course_type == LAB_CLASS and day1 != day2:
            conflicts += 1
        # 9. One professor teaches only one course section at the same time on the same day
        if (professor_id, day1) in professor_schedule.values() or (professor_id, day2) in professor_schedule.values():
            conflicts += 1

        # Update schedules
        professor_schedule[(professor_id, day1)] = room_number
        professor_schedule[(professor_id, day2)] = room_number
        room_schedule[(day1, time_slot_index1)] = (course_type, room_number, room_capacity)
        room_schedule[(day2, time_slot_index2)] = (course_type, room_number, room_capacity)
        section_schedule[(day1, room_number)] = course_type
        section_schedule[(day2, room_number)] = course_type
        daily_schedule[day1][time_slot_index1].append((course_id, section_id))
        daily_schedule[day2][time_slot_index2].append((course_id, section_id))

    fitness = -conflicts
    return fitness


# Example of generating initial population
population_size = 10
population = [chromosome_generation() for _ in range(population_size)]

# Define crossover function
def crossover(parent1, parent2):
    # the parents are converted to chromosomes
    chromosomes1 = parent1
    chromosomes2 = parent2
    # 2 point crossover is performed
    crossover_point1 = random.randint(0, len(chromosomes1) - 1)
    crossover_point2 = random.randint(0, len(chromosomes1) - 1)
    if crossover_point1 > crossover_point2:
        crossover_point1, crossover_point2 = crossover_point2, crossover_point1
    child1 = chromosomes1[:crossover_point1] + chromosomes2[crossover_point1:crossover_point2] + chromosomes1[
                                                                                                         crossover_point2:]
    child2 = chromosomes2[:crossover_point1] + chromosomes1[crossover_point1:crossover_point2] + chromosomes2[
                                                                                                         crossover_point2:]
    return child1, child2

# Find indices of two maximum fitness scores
def selection(population):
    fitness_scores = [fitness_Calculation(chromosome) for chromosome in population]
    max_fitness_indices = sorted(range(len(fitness_scores)), key=lambda i: fitness_scores[i])[-2:]
    return [population[i] for i in max_fitness_indices]

# Calculate fitness for the entire population
def calculate_fitness_population(population):
    return [fitness_Calculation(chromosome) for chromosome in population]

def mutation(chromosome, mutation_rate):
    mutated_chromosome = []
    for course_info in chromosome:
        mutated_course_info = list(course_info)  # Make a copy of the course info
        # Apply mutation to the binary attributes
        for i in range(len(mutated_course_info) - 2):  # Skip the last two elements which are not binary
            if random.random() < mutation_rate:
                # Mutate the bit
                mutated_course_info[i] = '1' if mutated_course_info[i] == '0' else '0'
        mutated_chromosome.append(tuple(mutated_course_info))  # Append the mutated course info to the chromosome
    return mutated_chromosome

# Main loop for the genetic algorithm
NUM_GENERATIONS = 10  # Number of generations
    # Same as before
for generation in range(NUM_GENERATIONS):
    print(f"Generation {generation + 1}:")
    fitness_scores = calculate_fitness_population(population)
    parent1, parent2 = selection(population)
    children = crossover(parent1, parent2)
    mutated_children = [mutation(child, MUTATION_RATE) for child in children]
    population.extend(mutated_children)
    population.sort(key=lambda x: fitness_Calculation(x), reverse=True)
    population = population[:population_size]
    best_fitness = max(fitness_scores)
    print(f"Best fitness score of generation {generation + 1}: {best_fitness}")

# Select the best chromosome after all generations
best_chromosome = max(population, key=lambda x: fitness_Calculation(x))
# Print the best timetable after all generations
print("\nBest timetable after all generations:")
for i, course_info in enumerate(best_chromosome):
    course_id = i + 1
    course_type, professor_id, day1, day2, room_number, room_capacity = course_info[2:]

    # Convert day1 and day2 to integers
    day1 = int(day1, 2)
    day2 = int(day2, 2)

    # Calculate time slot indices
    time_slot_index1 = day1 * NUM_TIME_SLOTS_PER_DAY // NUM_DAYS
    time_slot_index2 = day2 * NUM_TIME_SLOTS_PER_DAY // NUM_DAYS

    time_slot1 = TIME_SLOTS[time_slot_index1]
    time_slot2 = TIME_SLOTS[time_slot_index2]

    # Create readable strings
    course_name = f"Course {course_id}"
    professor_name = f"Professor {str(int(professor_id, 2) + 1)}"  # Convert professor_id to integer before addition
    room_name = f"Room {str(int(room_number, 2) + 1)}"  # Convert room_number to integer before addition

    # Create timetable entries
    timetable_entry1 = f"Day {day1 + 1}, {time_slot1[0]} - {time_slot1[1]}: {course_name} ({course_type}), {professor_name}, {room_name}"
    timetable_entry2 = f"Day {day2 + 1}, {time_slot2[0]} - {time_slot2[1]}: {course_name} ({course_type}), {professor_name}, {room_name}"

    print(timetable_entry1)
    print(timetable_entry2)
    print()


Generation 1:
Best fitness score of generation 1: -195
Generation 2:
Best fitness score of generation 2: -195
Generation 3:
Best fitness score of generation 3: -195
Generation 4:
Best fitness score of generation 4: -193
Generation 5:
Best fitness score of generation 5: -193
Generation 6:
Best fitness score of generation 6: -193
Generation 7:
Best fitness score of generation 7: -193
Generation 8:
Best fitness score of generation 8: -191
Generation 9:
Best fitness score of generation 9: -191
Generation 10:
Best fitness score of generation 10: -189

Best timetable after all generations:
Day 3, 11:40 - 1:00: Course 1 (Theory), Professor 8, Room 2
Day 5, 2:50 - 4:10: Course 1 (Theory), Professor 8, Room 2

Day 2, 10:05 - 11:25: Course 2 (Theory), Professor 7, Room 11
Day 4, 1:15 - 2:35: Course 2 (Theory), Professor 7, Room 11

Day 4, 1:15 - 2:35: Course 3 (Lab), Professor 8, Room 3
Day 1, 8:30 - 9:50: Course 3 (Lab), Professor 8, Room 3

Day 4, 1:15 - 2:35: Course 4 (Lab), Professor 2, Room